## Simulating Radiances

This example covers the simualation of synthetic measurements using pyshdom. This is a simple and streamlined example.

In [1]:
#imports

import pyshdom
import numpy as np
import xarray as xr
from collections import OrderedDict
import pylab as py

### Make the RTE grid and medium microphysics.

This time we are going to include both cloud and rayleigh scatter. 
The fundamental thing that must be defined first is the `rte_grid`, which sets the domain size. We have an easy choice here from the medium but in the inverse problem this must be selected as the initial step.

In [2]:
cloud_scatterer = pyshdom.util.load_from_csv('../data/synthetic_cloud_fields/jpl_les/rico32x37x26.txt',
                                           density='lwc',origin=(0.0,0.0))

#load atmosphere
atmosphere = xr.open_dataset('../data/ancillary/AFGL_summer_mid_lat.nc')
#subset the atmosphere, choose only the bottom four km.
reduced_atmosphere = atmosphere.sel({'z': atmosphere.coords['z'].data[atmosphere.coords['z'].data <= 4.0]})
#merge the atmosphere and cloud z coordinates
merged_z_coordinate = pyshdom.grid.combine_z_coordinates([reduced_atmosphere,cloud_scatterer])

# define the property grid - which is equivalent to the base RTE grid
rte_grid = pyshdom.grid.make_grid(cloud_scatterer.x.diff('x')[0],cloud_scatterer.x.data.size,
                          cloud_scatterer.y.diff('y')[0],cloud_scatterer.y.data.size,
                          cloud_scatterer.z.data)


In [3]:
#finish defining microphysics because we can.

cloud_scatterer_on_rte_grid = pyshdom.grid.resample_onto_grid(rte_grid, cloud_scatterer)

#We choose a gamma size distribution and therefore need to define a 'veff' variable.
size_distribution_function = pyshdom.size_distribution.gamma

cloud_scatterer_on_rte_grid['veff'] = (cloud_scatterer_on_rte_grid.reff.dims,
                                       np.full_like(cloud_scatterer_on_rte_grid.reff.data, fill_value=0.1))

### Define Sensors

This is based on AirMSPI - like measurements. 9 observations in an arc.

and eMAS observations - nadir multi-spectral measurements.

In [4]:
sensor_dict = pyshdom.containers.SensorsDict()

In [5]:
sensor_zenith_list = [75.0,60.0,45.6,26.1]*2 + [0.0]
sensor_azimuth_list = [90]*4 + [-90]*4 +[0.0]
for zenith,azimuth in zip(sensor_zenith_list,sensor_azimuth_list):
    sensor_dict.add_sensor('MSPI', 
            pyshdom.sensor.orthographic_projection(0.86, cloud_scatterer,0.02,0.02, azimuth, zenith,
                                     altitude='TOA', stokes='I')
                          )

In [6]:
# for wavelength in [1.65, 2.2, 3.7]:
#     sensor_dict.add_sensor('eMAS', 
#             pyshdom.sensor.orthographic_projection(wavelength, cloud_scatterer,0.02,0.02, azimuth, zenith,
#                                      altitude='TOA', stokes='I')
#                           )

In [7]:
wavelengths = sensor_dict.get_unique_solvers()

### get optical property generators

Note that we assume a monochromatic model here.

In [8]:
mie_mono_tables = OrderedDict()
for wavelength in wavelengths:
    mie_mono_tables[wavelength] = pyshdom.mie.get_mono_table(
        'Water',(wavelength,wavelength),
        max_integration_radius=65.0,
        minimum_effective_radius=0.1,
        relative_dir='../mie_tables',
        verbose=False
    )

In [9]:
optical_property_generator = pyshdom.medium.OpticalPropertyGenerator(
    'cloud', 
    mie_mono_tables,
    size_distribution_function,
    reff=np.linspace(5.0,30.0,30),
    veff=np.linspace(0.03,0.2,9),
)
optical_properties = optical_property_generator(cloud_scatterer_on_rte_grid)

In [10]:
# one function to generate rayleigh scattering.
rayleigh_scattering = pyshdom.rayleigh.to_grid(wavelengths,atmosphere,rte_grid)

## Define Solvers

Define solvers last based on the sensor's spectral information.

In [11]:
solvers_dict = pyshdom.containers.SolversDict()
# note we could set solver dependent surfaces / sources / numerical_config here
# just as we have got solver dependent optical properties.

for wavelength in sensor_dict.get_unique_solvers():
    medium = {
        'cloud': optical_properties[wavelength],
        #'rayleigh':rayleigh_scattering[wavelength]
     }
    config = pyshdom.configuration.get_config('../default_config.json')
    #config['num_mu_bins'] = 2
    #config['num_phi_bins'] = 4
    solvers_dict.add_solver(
        wavelength,
        pyshdom.solver.RTE(
            numerical_params=config,
            surface=pyshdom.surface.lambertian(0.0),
            source=pyshdom.source.solar(wavelength, 0.5,0.0),
            medium=medium,
            num_stokes=1#sensor_dict.get_minimum_stokes()[wavelength],
        )                   
   )

In [12]:
%%time
# solve the 4 RTEs in parallel AND get the measurements.
sensor_dict.get_measurements(solvers_dict, n_jobs=1)

CPU times: user 51.7 s, sys: 259 ms, total: 52 s
Wall time: 52.2 s
CPU times: user 51.7 s, sys: 259 ms, total: 52 s
Wall time: 52.2 s


### Visualize the observations


In [ ]:
for image in sensor_dict.get_images('MSPI'):
    py.figure()
    image.I.plot()

In [13]:
pyshdom.util.save_forward_model('SimulateRadiances.nc', sensor_dict, solvers_dict)